In [1]:
from typing import Union, Tuple, Optional
import bw2data as bd
import bw2calc as bc
import bw_processing as bwp
import uuid
import logging
import numpy as np

Change level to `logging.DEBUG` to print too much, `logging.WARNING` to print less

In [2]:
logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger('shaving-club')
logger.setLevel(level=logging.INFO)

## Moving an edge to another producing node safely

We use datapackages so that the underlying database isn't modified.

In [3]:
def safety_razor(
        consumer: Union[bd.Node, Tuple[str, str], int], 
        previous_producer: Union[bd.Node, Tuple[str, str], int], 
        new_producer: Union[bd.Node, Tuple[str, str], int], 
        datapackage: Optional[bwp.Datapackage] = None,
        amount: Optional[float] = None,
        name: Optional[str] = None,
    ) -> bwp.Datapackage:
    """Replace an existing edge with another edge. Zeroes out the existing edge.

    Inputs:
    consumer: Union[bd.Node, Tuple[str, str], int]
        The consuming node 
    previous_producer: Union[bd.Node, Tuple[str, str], int]
        The producing node which should be replaced
    new_producer: Union[bd.Node, Tuple[str, str], int]
        The new producing node
    datapackage: Optional[bwp.Datapackage]
        Append to this datapackage, if available. Otherwise create a new datapackage.
    amount: Optional[float]
        Amount of the new edge. Will be the *sum of all (previous_producer, consumer) edge amounts if not provided.
    name: Optional[str]
        Name of this datapackage resource.
    
    Returns a `bw_processing.Datapackage` with the modified data."""

    def resolve_node(node: Union[bd.Node, Tuple[str, str], int]) -> bd.Node:
        """Return a Brightway node from many different input possibilities.
        
        This isn't super-efficient - you could look up the `id` values ahead of time.
        In production you don't need fancy logging messages."""
        if isinstance(node, tuple):
            assert len(node) == 2
            return bd.get_node(database=node[0], code=node[1])
        elif isinstance(node, int):
            return bd.get_node(id=int)
        elif isinstance(node, bd.Node):
            return node
        else:
            raise ValueError(f"Can't understand {node}")
                
    consumer = resolve_node(consumer)
    previous_producer = resolve_node(previous_producer)
    new_producer = resolve_node(new_producer)

    assert new_producer.get("type", "process") == "process", "Wrong type of edge source"
    # Remove if creating new edge instead of moving or replacing existing an edge
    assert any(exc.input == previous_producer for exc in consumer.technosphere())

    if not name:
        name = uuid.uuid4().hex
        logger.info(f"Using random name {name}")

    if not amount:
        amount = sum(
            exc['amount'] 
            for exc in consumer.technosphere() 
            if exc.input == previous_producer
        )
        logger.info(f"Using database net amount {amount}")

    logger.info(f"Zeroing exchange from {previous_producer} to {consumer}")
    logger.info(f"Adding exchange of {amount} {new_producer} to {consumer}")

    if datapackage is None:
        datapackage = bwp.create_datapackage()

    datapackage.add_persistent_vector(
        # This function would need to be adapted for biosphere edges
        matrix="technosphere_matrix",
        name=name,
        data_array=np.array([0, amount], dtype=float),
        indices_array=np.array([
                (previous_producer.id, consumer.id), 
                (new_producer.id, consumer.id)
            ], dtype=bwp.INDICES_DTYPE),
        flip_array=np.array([False, True], dtype=bool)
    )  
    return datapackage

## Usage

In [4]:
import bw2io as bi
import bw2analyzer as ba

In [6]:
bi.restore_project_directory(
    fp="/srv/data/ecoinvent-3.9-cutoff.tar.gz", 
    project_name="🪒",  # Some silliness late at night :)
    overwrite_existing=True
)

Restoring project backup archive - this could take a few minutes...


'🪒'

In [5]:
bd.projects.set_current("🪒")

In [6]:
pear_market = bd.get_node(name="market for pear")
pear_china = bd.get_node(name="pear production", location="CN")
apple = bd.get_node(name="apple production", location="CL")

In [7]:
ipcc = ('IPCC 2021', 'climate change: fossil', 'global warming potential (GWP100)')

In [8]:
demand = {pear_market: 1}

Get the list of `data_objs` - our new datapackage will be appended to this list.

In [10]:
fu, data_objs, remapping = bd.prepare_lca_inputs(demand=demand, method=ipcc)

In [13]:
lca = bc.LCA(fu, data_objs=data_objs, remapping_dicts=remapping)
lca.lci()
lca.lcia()
print("Pear:", lca.score)

Pear: 0.44607599419190447


In [15]:
dp = safety_razor(
    consumer=pear_market,
    previous_producer=pear_china, 
    new_producer=apple, 
)

INFO:shaving-club:Using random name 054616497092409c876e1c722aeb7bef
INFO:shaving-club:Using database net amount 0.693363752738162
INFO:shaving-club:Zeroing exchange from 'pear production' (kilogram, CN, None) to 'market for pear' (kilogram, GLO, None)
INFO:shaving-club:Adding exchange of 0.693363752738162 'apple production' (kilogram, CL, None) to 'market for pear' (kilogram, GLO, None)


In [16]:
lca = bc.LCA(fu, data_objs=data_objs + [dp], remapping_dicts=remapping)
lca.lci()
lca.lcia()
print("Apples are not pears:", lca.score)

Apples are not pears: 0.25399614413937016
